# Пакеты `pandas` и `numpy`: типы данных


---

**Источники:**


[what are all the dtypes that pandas recognizes?](https://stackoverflow.com/questions/29245848/what-are-all-the-dtypes-that-pandas-recognizes)

[numpy.dtype](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html#numpy.dtype)

[Data types](https://numpy.org/doc/stable/user/basics.types.html)

[Data type objects (dtype)](https://numpy.org/doc/stable/reference/arrays.dtypes.html#arrays-dtypes)

[Scalars](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.scalars.html)

[Categorical data](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)

[Essential basic functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html)

[Extension types](https://pandas.pydata.org/pandas-docs/stable/development/extending.html#extending-extension-types)

---

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

!pip -V

pip 20.3.3 from /home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
# !conda install pandas numpy lxml -y

In [3]:
import numpy as np

np.__version__

'1.19.2'

In [4]:
import pandas as pd

pd.__version__

'1.2.1'

## Обзор типов данных

**`Pandas` в основном использует массивы и типы `NumPy`** для каждой серии (фрейм данных - это набор серий, каждая из которых может иметь свой собственный `dtype`).

Документация `NumPy` дополнительно объясняет [`dtype`](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html#numpy.dtype), [`Data types`](https://numpy.org/doc/stable/user/basics.types.html) и [`Data type objects (dtype)`](https://numpy.org/doc/stable/reference/arrays.dtypes.html#arrays-dtypes).


In [5]:
df = pd.DataFrame({'A': [1, 'C', 2.]})

df

A
0    1
1    C
2  2.0

In [6]:
df['A']

0      1
1      C
2    2.0
Name: A, dtype: object

In [7]:
df['A'].dtype

dtype('O')

In [8]:
type(df['A'].dtype)

numpy.dtype

По умолчанию `int` - это `int64`, а `float` - `float64`, НЕЗАВИСИМО от платформы (32-разрядная или 64-разрядная).

In [9]:
pd.DataFrame([1, 2], columns=["a"]).dtypes

a    int64
dtype: object

In [10]:
pd.DataFrame({"a": [1, 2]}).dtypes

a    int64
dtype: object

In [11]:
pd.DataFrame({"a": 1}, index=list(range(2))).dtypes

a    int64
dtype: object

`Numpy`, однако, выберет платформо-зависимые типы *при создании массивов*. Следующее будет результатом int32 на 32-битной платформе.

In [12]:
pd.DataFrame(np.array([1, 2])).dtypes

0    int64
dtype: object

Одним из основных изменений в версии 1.0.0 `pandas` является введение `pd.NA` для представления скалярных пропущенных значений (а не предыдущих значений `np.nan`, `pd.NaT` или `None`, в зависимости от использования).

## Пакет `numpy`: типы данных

<img src="images/dtype_hierarchy.png"/>

In [13]:
def subdtypes(dtype):
    subs = dtype.__subclasses__()
    if not subs:
        return dtype
    return [dtype, [subdtypes(dt) for dt in subs]]

subdtypes(np.generic)

[numpy.generic,
 [[numpy.number,
   [[numpy.integer,
     [[numpy.signedinteger,
       [numpy.int8,
        numpy.int16,
        numpy.int32,
        numpy.int64,
        numpy.longlong,
        numpy.timedelta64]],
      [numpy.unsignedinteger,
       [numpy.uint8,
        numpy.uint16,
        numpy.uint32,
        numpy.uint64,
        numpy.ulonglong]]]],
    [numpy.inexact,
     [[numpy.floating,
       [numpy.float16, numpy.float32, numpy.float64, numpy.float128]],
      [numpy.complexfloating,
       [numpy.complex64, numpy.complex128, numpy.complex256]]]]]],
  [numpy.flexible,
   [[numpy.character, [numpy.bytes_, numpy.str_]],
    [numpy.void, [numpy.record]]]],
  numpy.bool_,
  numpy.datetime64,
  numpy.object_]]

In [14]:
np.float32(1.0)

1.0

In [15]:
np.int_([1,2,4])

array([1, 2, 4])

In [16]:
z = np.arange(3, dtype=np.uint8)
z

array([0, 1, 2], dtype=uint8)

Чтобы преобразовать тип массива, используйте метод `.astype()` (предпочтительно) или сам тип как функцию.

In [17]:
# предпочтительно
z = z.astype(float)  
z

array([0., 1., 2.])

In [18]:
z.dtype

dtype('float64')

In [19]:
z = np.complex_(z)
z

array([0.+0.j, 1.+0.j, 2.+0.j])

In [20]:
z.dtype

dtype('complex128')

In [21]:
d = np.dtype(int)
d

dtype('int64')

In [22]:
np.issubdtype(d, np.integer)

True

In [23]:
np.issubdtype(d, np.floating)

False

### [Ошибки переполнения (Overflow Errors)](https://numpy.org/doc/stable/user/basics.types.html#overflow-errors)

Поведение целочисленных типов `NumPy` и `Python` значительно отличается для целочисленных переполнений и может сбить с толку пользователей, ожидающих, что целые числа `NumPy` будут вести себя так же, как `int` в `Python`.

В отличие от `NumPy`, размер `int` в `Python` является гибким. Это означает, что **целые числа `Python` могут расширяться для размещения любого целого числа и не будут переполняться**.

**Фиксированный размер числовых типов `NumPy` может вызвать ошибки переполнения**, когда значение требует больше памяти, чем доступно в типе данных.

In [24]:
np.power(100, 8, dtype=np.int64)

10000000000000000

In [25]:
np.power(100, 8, dtype=np.int32)

1874919424

`NumPy` предоставляет `numpy.iinfo` и `numpy.finfo` для проверки минимального или максимального значений целых чисел и значений с плавающей запятой `NumPy` соответственно.

In [26]:
np.iinfo(int)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [27]:
np.iinfo(np.int64)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [28]:
np.iinfo(np.int32)

iinfo(min=-2147483648, max=2147483647, dtype=int32)

In [29]:
np.power(100, 100, dtype=np.int64)

0

In [30]:
np.power(100, 100, dtype=np.float64)

1e+200

## Пакет `Pandas`: типы данных

По большей части `pandas` использует массивы и типы `NumPy` для серий или отдельных столбцов `DataFrame`. 

`NumPy` обеспечивает поддержку `float`, `int`, `bool`, `timedelta64[ns]` и `datetime64[ns]` (обратите внимание, что `NumPy` не поддерживает дату и время с учетом часового пояса).

`pandas` и сторонние (third-party) библиотеки расширяют систему типов `NumPy` в нескольких местах.

В этом разделе описываются внутренние расширения `pandas`.

В следующей таблице перечислены все типы расширений `pandas`.

In [31]:
htmp_basic_info = 'https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes'
basic_info_tables = pd.read_html(htmp_basic_info)

for index, table in enumerate(basic_info_tables):
    print(f"index = {index}")
    display(table)

In [32]:
pandas_types_df = basic_info_tables[3]

pandas_types_df

Kind of Data         Data Type     Scalar                 Array  \
0    tz-aware datetime   DatetimeTZDtype  Timestamp  arrays.DatetimeArray   
1          Categorical  CategoricalDtype     (none)           Categorical   
2  period (time spans)       PeriodDtype     Period    arrays.PeriodArray   
3               sparse       SparseDtype     (none)    arrays.SparseArray   
4            intervals     IntervalDtype   Interval  arrays.IntervalArray   
5     nullable integer     Int64Dtype, …     (none)   arrays.IntegerArray   
6     nullable integer     Int64Dtype, …     (none)   arrays.IntegerArray   
7              Strings       StringDtype        str    arrays.StringArray   
8    Boolean (with NA)      BooleanDtype       bool   arrays.BooleanArray   

                                      String Aliases  \
0                             'datetime64[ns, <tz>]'   
1                                         'category'   
2                 'period[<freq>]', 'Period[<freq>]'   
3           'Sparse', 'Sparse[int]', 'Sparse[float]'   
4  'interval', 'Interval', 'Interval[<numpy_dtype...   
5  'Int8', 'Int16', 'Int32', 'Int64', 'UInt8', 'U...   
6  'Int8', 'Int16', 'Int32', 'Int64', 'UInt8', 'U...   
7                                           'string'   
8                                          'boolean'   

                      Documentation  
0                Time zone handling  
1                  Categorical data  
2          Time span representation  
3            Sparse data structures  
4                     IntervalIndex  
5        Nullable integer data type  
6        Nullable integer data type  
7            Working with text data  
8  Boolean data with missing values

### [Обработка часовых поясов (Time zone handling)](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-timezone)

**Вид данных:** 

Дата и время с учетом часового пояса (`tz`) (обратите внимание, что `NumPy` не поддерживает дату и время с учетом часовых поясов).

`pandas` предоставляет хорошую поддержку работы с отметками времен (timestamps) и в разных часовых поясах с использованием библиотек `pytz` и `dateutil` или объектов `datetime.timezone` из стандартной библиотеки.

### [Категориальные данные (Categorical data)](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical)

**Вид данных:** 

Категории - это тип данных, соответствующий категориальным переменным в статистике. 

Категориальная переменная принимает ограниченное и обычно фиксированное количество возможных значений.

Примеры: пол, социальный класс, группа крови, принадлежность к стране, язык.

### [Представление промежутка времени (Time span representation)](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-periods)

**Вид данных:** 

Период представляет собой промежуток времени (например, день, месяц, квартал и т. Д.).

Можно указать диапазон с помощью ключевого слова `freq`.

Обычные интервалы времени представлены объектами `Period` в `pandas`, в то время как последовательности объектов `Period` собираются в `PeriodIndex`, который можно создать с помощью удобной функции `period_range`.

### [Разреженные структуры данных (Sparse data structures)](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#sparse)

**Вид данных:**

Различные разреженные матрицы.

Это не обязательно разреженность в типичном варианте: "в основном 0".

Скорее, можно рассматривать эти объекты как "сжатые", где любые данные, соответствующие определенному значению (`NaN` / отсутствующее значение, хотя может быть выбрано любое значение, включая `0`), опущены.

Сжатые значения фактически не хранятся в массиве.

### [Интервальный индекс (IntervalIndex)](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-intervalindex)

**Вид данных:**

`IntervalIndex` вместе со своим собственным `dtype` - `IntervalDtype`, а также скалярным типом `Interval` обеспечивают первоклассную поддержку `pandas` для обозначения интервалов.


### [Обнуляемый целочисленный тип данных (Nullable integer data type)](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html#integer-na)

**Вид данных:** 

`pandas` в основном использует `NaN` для представления отсутствующих данных.

Поскольку `NaN` является `float`, это заставляет массив целых чисел (`int`) с любыми пропущенными значениями стать `float`.

В некоторых случаях это может не иметь большого значения.

Но если целочисленный столбец является, например, идентификатором, приведение к типу `float` может быть проблематичным. 

Некоторые целые числа даже нельзя представить как числа с плавающей запятой.

### [Текстовые типы данных (Text data types)](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)

**Вид данных:** 

Есть два способа хранить текстовые данные в `pandas`: 
- объект `dtype` `NumPy` `array`.
- тип `StringDtype`.


`pandas` рекомендует использовать `StringDtype` для хранения текстовых данных.

### [Логические данные с пропущенными значениями (Boolean data with missing values)](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html#api-arrays-bool)

**Вид данных:** 

Логический тип `dtype` (с псевдонимом `boolean`) обеспечивает поддержку для хранения логических данных (значения `True`, `False`) с пропущенными значениями, что невозможно с логическим значением `numpy.ndarray`.